In [1]:
def run_linux_command(command_list, stdin_file=None, stdout_file=None):
    if platform.system() in ["Linux", "Darwin"]:
        # If we are in linux or mac os, we can directly execute the commands
        subprocess.run(command_list,
                       stdin=open(stdin_file) if stdin_file is not None else None,
                       stdout=open(stdout_file, "w") if stdout_file is not None else None)
        
def apply_volume_registration_to_points(registration_method, transform_list, data_points):
    print(datetime.datetime.now(), "Running apply_volume_registration_to_points.", locals())
    # create some random number, to avoid parallel processes with the same filenames
    all_data_points_path = Path(tempfile.gettempdir()) / f"{str(uuid.uuid4())}_all_data_points.csv"
    all_data_points_registered_path = Path(tempfile.gettempdir()) / f"{str(uuid.uuid4())}_all_data_points_registered.csv"
    print(all_data_points_path)
    print(all_data_points_registered_path)
    all_data_points_path_linux = convert_path_to_linux(all_data_points_path)
    all_data_points_registered_linux = convert_path_to_linux(all_data_points_registered_path)
    if registration_method == "ants":
        # Ants wants four columns
        data_points_ants = np.c_[data_points, np.zeros(data_points.shape[0])]
        np.savetxt(all_data_points_path, data_points_ants, delimiter=',', header="x,y,z,t", comments='')
        registration_commands_list = [f"{ants_bin_path}/antsApplyTransformsToPoints",
                                      "--precision", "0",   # float32 is enough
                                      "--dimensionality", "3",
                                      "--input", f"{all_data_points_path_linux}",
                                      "--output", f"{all_data_points_registered_linux}"]
        for transform in transform_list:
            registration_files_prefix_linux = convert_path_to_linux(transform)
            registration_commands_list += ["--transform", transform]
        # if not inverted, use the inverted matrix for the computation (because of points)
        #
        # if use_inverted_transforms is False:
        #     for registration_files_prefix in registration_files_prefix_list[::-1]:
        #         registration_files_prefix_linux = convert_path_to_linux(registration_files_prefix)
        #
        #         registration_commands_list += ["--transform", f"[{registration_files_prefix_linux}_0GenericAffine.mat, 1]",
        #                                        "--transform", f"{registration_files_prefix_linux}_1InverseWarp.nii.gz"]
        # else:
        #     for registration_files_prefix in registration_files_prefix_list:
        #         registration_files_prefix_linux = convert_path_to_linux(registration_files_prefix)
        #
        #         registration_commands_list += ["--transform", f"{registration_files_prefix_linux}_1Warp.nii.gz",
        #                                        "--transform", f"{registration_files_prefix_linux}_0GenericAffine.mat"]
        print(registration_commands_list)
        run_linux_command(registration_commands_list)
        transformed_points = np.loadtxt(all_data_points_registered_path, delimiter=',', skiprows=1, usecols=(0, 1, 2))
    if registration_method == "cmtk":
        np.savetxt(all_data_points_path, data_points, delimiter=' ')
        registration_commands_list = [f"{cmtk_bin_path}/streamxform"]   # streamxform uses only one thread
        for transform in transform_list:
            registration_commands_list += [transform]
        #
        # # if not inverted, use the inverted matrix for the computation (because of points)
        # # TODO remove the inverted also for CMTK, the user should specifiy this
        # if use_inverted_transforms is False:
        #     for registration_files_prefix in registration_files_prefix_list[::-1]:
        #         registration_files_prefix_linux = convert_path_to_linux(registration_files_prefix)
        #
        #         registration_commands_list += ["--",
        #                                        "--inverse", f"{registration_files_prefix_linux}_ffd5_transformation.xform"]
        # else:
        #     for registration_files_prefix in registration_files_prefix_list:
        #         registration_files_prefix_linux = convert_path_to_linux(registration_files_prefix)
        #
        #         registration_commands_list += [f"{registration_files_prefix_linux}_ffd5_transformation.xform"]
        run_linux_command(registration_commands_list,
                          stdin_file=all_data_points_path_linux,
                          stdout_file=all_data_points_registered_linux)
        transformed_points = np.loadtxt(all_data_points_registered_path, delimiter=' ', usecols=(0, 1, 2))
    # file = open(registration_commands_path, 'wb')
    # file.write((registration_commands + '\n').encode())
    # file.close()
    #
    # #####################
    # print("Executing ubuntu script ....")
    # print(registration_commands)
    #
    # sp.call(["bash", '-c', registration_commands_path_linux])
    all_data_points_path.unlink()
    all_data_points_registered_path.unlink()
    return transformed_points

def apply_volume_registration_to_swc(registration_method, transform_list,
                                     cell_input_path, cell_output_path,
                                     node_size_scale,
                                     input_dx, input_dy, input_dz,
                                     output_dx, output_dy, output_dz,
                                     input_flip_x=None, input_flip_y=None, input_flip_z=None,
                                     output_flip_x=None, output_flip_y=None, output_flip_z=None,
                                     input_skiprows=1, skip_if_exists=True):
    print(datetime.datetime.now(), "Running apply_volume_registration_to_swc.", locals())
    if skip_if_exists and cell_output_path.exists():
        print(f"Cell {cell_input_path} has already been registered.")
        return
    cell_data = np.loadtxt(cell_input_path, skiprows=input_skiprows, delimiter=' ')
    if input_flip_x != None:
        cell_data[:, 2] = input_flip_x - cell_data[:, 2]
    if input_flip_y != None:
        cell_data[:, 3] = input_flip_y - cell_data[:, 3]
    if input_flip_z != None:
        cell_data[:, 4] = input_flip_z - cell_data[:, 4]
    data_points = np.c_[cell_data[:, 2] * input_dx,
                        cell_data[:, 3] * input_dy,
                        cell_data[:, 4] * input_dz]
    data_points_transformed = apply_volume_registration_to_points(registration_method,
                                                                  transform_list,
                                                                  data_points) # don't the inverted thing anymore, the user must specificy this!
    # the x_scale is computes as the ratio of target and source resolution
    transformed_cell_data = np.c_[cell_data[:, 0], cell_data[:, 1],
                                  data_points_transformed[:, 0] / output_dx,
                                  data_points_transformed[:, 1] / output_dy,
                                  data_points_transformed[:, 2] / output_dz,
                                  cell_data[:, 5] * node_size_scale,
                                  cell_data[:, 6]]
    if output_flip_x != None:
        transformed_cell_data[:, 2] = output_flip_x - transformed_cell_data[:, 2]
    if output_flip_y != None:
        transformed_cell_data[:, 3] = output_flip_y - transformed_cell_data[:, 3]
    if output_flip_z != None:
        transformed_cell_data[:, 4] = output_flip_z - transformed_cell_data[:, 4]
    df = pandas.DataFrame(transformed_cell_data, columns=['index', 'neuronname', 'x', 'y', 'z', 'size', 'connects'])
    df["index"] = df["index"].astype(np.int)
    df["neuronname"] = df["neuronname"].astype(np.int)
    df["connects"] = df["connects"].astype(np.int)
    df.to_csv(cell_output_path, index=False, sep=' ')